In [3]:
import re
import string
import numpy as np
import newspaper


In [4]:
# Import required module


# Assign url
urls = ['https://www.geeksforgeeks.org/top-5-open-source-online-machine-learning-environments/',
        'https://www.google.com/url?sa=t&rct=j&q=&esrc=s&source=web&cd=&ved=2ahUKEwjI56vC3MH9AhUP8zgGHbryB0sQvOMEKAB6BAgOEAE&url=https%3A%2F%2Fwww.thehindu.com%2Fnews%2Finternational%2Fquad-foreign-ministers-take-aim-at-russia-and-china%2Farticle66577074.ece&usg=AOvVaw3G125a3AJkzEiMjMpC3Tt_',
        'https://www.thehindu.com/news/national/better-infrastructure-has-put-remote-indian-villages-on-tourist-map-pm-modi/article66575058.ece',
        'https://www.thehindu.com/sci-tech/health/avoid-antibiotics-for-seasonal-cold-and-cough-says-indian-medical-association-amid-rising-cases/article66579326.ece',
        ]

# Extract web data
articles = []
for url in urls:
    article = newspaper.Article(url="%s" % (url), language='en')
    article.download()
    article.parse()
    articles.append({
        'title': article.title,
        'text': article.text
    })

# Display scrapped data
#articles[0]


In [5]:

import requests
from bs4 import BeautifulSoup

input_words = ['atm', 'Machine Learning', 'theft', 'party']
word = 'theft'
url = f'https://www.thesaurus.com/browse/{word}'

response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

synonyms_div = soup.find('ul', {'class': 'css-x6mqrn e1ccqdb60'})#css-1kg1yv8 eh475bn0 css-1wgnrap e1ccqdb60 css-ttoca5 e1ccqdb60
synonyms_list = synonyms_div.find_all('a')
synonyms = [s.text.strip() for s in synonyms_list]#.find_all('a')]

print(f"Synonyms of {word}: {synonyms}")

Synonyms of theft: ['break in', 'burglary', 'crime', 'embezzlement', 'extortion', 'fraud', 'heist', 'holdup', 'larceny', 'looting', 'mugging', 'piracy', 'robbery', 'shoplifting', 'thievery', 'vandalism', 'annexation', 'appropriation', 'caper', 'cheating', 'deprivation', 'grab', 'hustle', 'job', 'lift', 'peculation', 'pilferage', 'pillage', 'pinch', 'plunder', 'racket', 'rapacity', 'ripoff', 'score', 'snatch', 'snitch', 'steal', 'stickup', 'swindle', 'thieving', 'touch', 'defrauding', 'pilfering', 'purloining', 'robbing', 'swindling']


In [6]:
news_list = articles
keywords_dict = {
    'atm': ['any time money', 'automatic teller machine'],
    'Machine Learning' : ['ml', 'ai'],
    'theft': ['robbery', 'steal', 'larceny'],
    'party': ['political', 'bjp', 'congress']
}  
news_list

[{'title': 'Top 5 Open-Source Online Machine Learning Environments',
  'text': 'Machine Learning is an area of research that allows machines the ability to learn without being directly programmed. Machine Learning development is in trend as many students, teachers, developers, and data scientists use machine learning to develop various projects and products. However, developing machine learning models require high system requirement specifications as sometimes the model training process can go from 2 hours to 2 days and more. So low-end systems can not handle training of good machine learning models or even if they somehow train models, critical system issues are likely to occur.\n\nHowever, there are many open-source Machine Learning environments available that do not require any system requirement specification and use cloud infrastructure to train your model in the most optimal time possible. Below are the most efficient, commonly used online machine learning environments:\n\nIt’s a

In [7]:

def removePunctuations(news_list):
    for i in range(len(news_list)):
        news_text = news_list[i]['text']
        news_list[i]['text'] = re.sub(r'[^\w\s]', '', news_text)
        news_title = news_list[i]['title']
        news_list[i]['title'] = re.sub(r'[^\w\s]', '', news_title)
    
    return news_list

In [8]:
heading_priority = 10
content_priority = 8

priority = list(np.arange(len(keywords_dict))[::-1]+1)


In [13]:
def getScore(words, content_type):
    score = 0
    frequency_map = {}
    for keyword in keywords_dict:
        frequency_map[keyword] = 0

    for i in range(len(words)):
        for j in range(len(keywords_dict)):
            # keyword = list(keywords_dict.keys())[j]
            # lis = [keyword]+keywords_dict[keyword]
            for synonym in keywords_list[j]:
                #print(keywords_list)
                if len(synonym.split(' ')) < 2:
                    if words[i].lower() == synonym.lower():
                        frequency_map[keywords_list[j][0]] = min(threshold, frequency_map[keywords_list[j][0]] + 1)
                else:
                    l = 0
                    index = i
                    while(l < len(synonym.split(' ')) and words[i].lower() == synonym.split(' ')[l].lower()):
                        i += 1
                        l += 1
                    if(l == len(synonym.split(' '))):
                        frequency_map[keywords_list[j][0]] = min(threshold, frequency_map[keywords_list[j][0]] + 1)
                    else:
                        i = index 
    
    index = 0
    for val in frequency_map.values():
        score += priority[index]*val
        index += 1
    #score += priority_score*sum(list(frequency_map.values()))
    if content_type == 'heading_priority':
        score += heading_priority*score
    else:
        score += content_priority*score
    print(frequency_map)
    return score

In [14]:
news_list = removePunctuations(news_list)
keywords_list = []
news_scores = []
threshold = 10

for j in range(len(keywords_dict)):
    keyword = list(keywords_dict.keys())[j]
    keywords_list.append([keyword]+keywords_dict[keyword])
    
for news in news_list:
    score = 0
    score += getScore(news['text'].split(' '), 'content_priority')
    score += getScore(news['title'].split(' '), 'heading_priority')   
    news_scores.append(score)

index = news_scores.index(max(news_scores))
# print(news_scores)
news_list[index]

{'atm': 0, 'Machine Learning': 10, 'theft': 0, 'party': 0}
{'atm': 0, 'Machine Learning': 1, 'theft': 0, 'party': 0}
{'atm': 0, 'Machine Learning': 0, 'theft': 0, 'party': 0}
{'atm': 0, 'Machine Learning': 0, 'theft': 0, 'party': 0}
{'atm': 0, 'Machine Learning': 0, 'theft': 0, 'party': 1}
{'atm': 0, 'Machine Learning': 0, 'theft': 0, 'party': 0}
{'atm': 0, 'Machine Learning': 0, 'theft': 0, 'party': 0}
{'atm': 0, 'Machine Learning': 0, 'theft': 0, 'party': 0}


{'title': 'Top 5 OpenSource Online Machine Learning Environments',
 'text': 'Machine Learning is an area of research that allows machines the ability to learn without being directly programmed Machine Learning development is in trend as many students teachers developers and data scientists use machine learning to develop various projects and products However developing machine learning models require high system requirement specifications as sometimes the model training process can go from 2 hours to 2 days and more So lowend systems can not handle training of good machine learning models or even if they somehow train models critical system issues are likely to occur\n\nHowever there are many opensource Machine Learning environments available that do not require any system requirement specification and use cloud infrastructure to train your model in the most optimal time possible Below are the most efficient commonly used online machine learning environments\n\nIts a cloud service that